In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

TARGET = 'target'

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,303.0,54.366337,9.082101,29.0,47.5,55.0,61.0,77.0
sex,303.0,0.683168,0.466011,0.0,0.0,1.0,1.0,1.0
cp,303.0,0.966997,1.032052,0.0,0.0,1.0,2.0,3.0
trestbps,303.0,131.623762,17.538143,94.0,120.0,130.0,140.0,200.0
chol,303.0,246.264026,51.830751,126.0,211.0,240.0,274.5,564.0
fbs,303.0,0.148515,0.356198,0.0,0.0,0.0,0.0,1.0
restecg,303.0,0.528053,0.525860,0.0,0.0,1.0,1.0,2.0
thalach,303.0,149.646865,22.905161,71.0,133.5,153.0,166.0,202.0
exang,303.0,0.326733,0.469794,0.0,0.0,0.0,1.0,1.0
oldpeak,303.0,1.039604,1.161075,0.0,0.0,0.8,1.6,6.2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
df.nunique().to_frame().reset_index().head()

,index,0
0,age,41
1,sex,2
2,cp,4
3,trestbps,49
4,chol,152


In [7]:
def extract_target(df, target_name):
    X = df.drop(target_name, axis=1)
    y = df[target_name]
    return X, y

In [8]:
def get_features_labels(X):

    cols_df = X.nunique().to_frame().reset_index()
    cols_df = cols_df.rename(columns={'index': 'column_name', 0: 'num_uniq'})

    all_feats = cols_df['column_name'].tolist()
    cat_feats = cols_df[cols_df['num_uniq']<10]['column_name'].tolist()
    num_feats = [x for x in all_feats if x not in cat_feats]
    
    return cat_feats, num_feats

In [9]:
def one_hot_transform(X, cat_feats):
    
    for i in cat_feats:
        X[i] = X[i].astype('str')

    one_hot_part = pd.get_dummies(X[cat_feats], prefix = cat_feats)
    return one_hot_part

In [10]:
def scaling_transform(X, num_feats):
    scaler = StandardScaler()
    scaling_part = pd.DataFrame(scaler.fit_transform(X[num_feats]))
    scaling_part.columns = num_feats
    
    return scaling_part

def full_transform(one_hot_part, scaling_part):
    full_transformed_df = pd.concat([one_hot_part,scaling_part], axis=1)
    
    return full_transformed_df

def train_model(X, y):
    model = LogisticRegression()
    model.fit(X, y)
    return model

In [11]:
X, y  = extract_target(df, TARGET)

cat_feats, num_feats = get_features_labels(X)

scal = scaling_transform(X, num_feats)
hot = one_hot_transform(X, cat_feats)

X_tr = full_transform(hot, scal)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
        X_tr, y, test_size=0.3, random_state=42
    )

In [13]:
model = train_model(X_train, y_train)

In [14]:
pred_labels = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:, 1]

In [15]:
result = {
        "accuracy": metrics.accuracy_score(y_test, pred_labels),
        "precision_score": metrics.precision_score(y_test, pred_labels),
        "recall": metrics.recall_score(y_test, pred_labels),
        "f1_score": metrics.f1_score(y_test, pred_labels),
        "roc_auc_score": metrics.roc_auc_score(y_test, pred_proba),
    }

In [16]:
result

{'accuracy': 0.8571428571428571,
 'precision_score': 0.8490566037735849,
 'recall': 0.9,
 'f1_score': 0.8737864077669903,
 'roc_auc_score': 0.915609756097561}